# Tree
Questo notebook serve per racchiudere quanto di utile per lo sviluppo di codice per valutare la scelta di alberi decisionali per il dataset scelto.

## Scaricamento dei dati

In [1]:
# File per scaricare i dati per fare analisi di machine learning

import requests
import zipfile
import io
import os

# List of URLs to your zipped files on AWS
urls = [
    "https://phm-datasets.s3.amazonaws.com/Data_Challenge_PHM2024_training_data.zip"]

# Directory to save the extracted files
output_dir = "dataset"
os.makedirs(output_dir, exist_ok=True)

for url in urls:
    try:
        print(f"Downloading {url}...")
        response = requests.get(url, stream=True)
        response.raise_for_status() # Raise an exception for bad status codes

        # Read the zip file from the response content
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            # Extract all contents to the specified output directory
            zip_ref.extractall(output_dir)
            print(f"Extracted files from {url} to {output_dir}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
    except zipfile.BadZipFile:
        print(f"Error: The downloaded file from {url} is not a valid zip file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

print("Download and extraction complete.")

# Now you can access your CSV files in the 'downloaded_data' directory
# For example, to list the files in the directory:
import glob
csv_files = glob.glob(os.path.join(output_dir, "*.csv"))
print("CSV files found:", csv_files)

Extracted files from https://phm-datasets.s3.amazonaws.com/Data_Challenge_PHM2024_training_data.zip to dataset
Download and extraction complete.
CSV files found: ['dataset/X_train.csv', 'dataset/y_train.csv']


## Estrazione dei dati

In [2]:
import pandas as pd

def load_data(x_path, y_path):
  """
  Loads X.csv and the second column of y.csv into a single pandas DataFrame.

  Args:
    x_path (str): The path to the X.csv file.
    y_path (str): The path to the y.csv file.

  Returns:
    pandas.DataFrame: A DataFrame containing the data from X.csv
                      and the second column of y.csv.
  """
  x = pd.read_csv(x_path)
  y = pd.read_csv(y_path)

  # Assuming y has at least 2 columns and the second column is at index 1
  if y.shape[1] > 1:
    combined_data = x.copy()
    combined_data['y_target'] = y.iloc[:, 1]
    return combined_data
  else:
    print("Error: y.csv does not have a second column.")
    return x

# Example usage:
# Assuming your files are in the 'dataset' directory as per the preceding code
x_path = 'dataset/X_train.csv'
y_path = 'dataset/y_train.csv'

data = load_data(x_path, y_path)

# You can now work with the 'data' DataFrame
print(data.head())


   id  trq_measured       oat       mgt         pa       ias         np  \
0   0        54.100   2.00000  544.5000   212.1408  74.56250   89.18000   
1   1        49.625  24.22231  578.4844  1625.6400  30.35596   99.55273   
2   2        52.000   7.00000  566.1000  1912.9250  65.62500  100.14000   
3   3        62.400   7.25000  560.1000   277.0632  54.81250   90.64000   
4   4        62.900  23.25000  593.7000    53.6448  73.43750   99.91000   

         ng  y_target  
0   99.6400         1  
1   91.3866         0  
2   90.9600         1  
3  100.2800         0  
4   92.1700         0  


## Creazione training-set testing-set

In [3]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

print("Dimensione data_train:", data_train.shape)
print("Dimensione data_test:", data_test.shape)

Dimensione data_train: (594100, 9)
Dimensione data_test: (148525, 9)


## Cambio nome delle feature

In [4]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted


# ---------------------- Custom Transformers ----------------------

class FeatureRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        self._is_fitted = True  # Mark as fitted
        return self

    def _rename_df(self, df):
        df = df.drop(columns='id', errors='ignore')
        if df.shape[1] != len(self.column_names):
            raise ValueError(f"Expected {len(self.column_names)} columns, got {df.shape[1]}")
        df.columns = self.column_names
        return df

    def transform(self, X):
        check_is_fitted(self, '_is_fitted')  # Check if fit was called
        return self._rename_df(X)

# ---------------------- Example Usage ----------------------

# Example column names
feature_names = [
    'torque_meas', 'outside_air_temp', 'mean_gas_temp',
    'power_avail', 'indicated_air_speed', 'net_power',
    'compressor_speed'
]

X_train = data_train.iloc[:, :-1]  # tutte le colonne tranne l'ultima → feature
y_train = data_train.iloc[:, -1]   # solo l'ultima colonna → target


pipeline = Pipeline([
    ('rename', FeatureRenamer(column_names=feature_names)),
])

pipeline.fit(X_train, y_train)

X_transformed = pipeline.transform(X_train)

print("Transformed DataFrame:")
print(X_transformed.head())

Transformed DataFrame:
        torque_meas  outside_air_temp  mean_gas_temp  power_avail  \
331067      74.8000            19.500       646.1000    1005.8400   
601458      67.0332            20.152       602.4063     958.4962   
77053       67.8000             8.000       546.7000      24.3840   
664037      53.9000             5.500       532.1000     360.8832   
346977      73.3000            17.750       636.3000    1025.9570   

        indicated_air_speed  net_power  compressor_speed  
331067             56.00000  100.19000          96.47000  
601458             96.97656   99.65235          94.02332  
77053              73.68750  100.22000          91.61000  
664037             61.87500   88.91000          99.78000  
346977             89.50000   99.98000          95.95000  


/home/tizianoc/.local/lib/python3.10/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


## Standardizzazione

In [5]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted


# ---------------------- Custom Transformers ----------------------

class FeatureRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        self._is_fitted = True  # Mark as fitted
        return self

    def _rename_df(self, df):
        df = df.drop(columns='id', errors='ignore')
        if df.shape[1] != len(self.column_names):
            raise ValueError(f"Expected {len(self.column_names)} columns, got {df.shape[1]}")
        df.columns = self.column_names
        return df

    def transform(self, X):
        check_is_fitted(self, '_is_fitted')  # Check if fit was called
        return self._rename_df(X)

class DataScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.columns_ = X.columns  # trailing underscore indicates fitted attribute
        self.scaler.fit(X)
        return self

    def transform(self, X):
        check_is_fitted(self.scaler, 'mean_')  # standard attribute for StandardScaler
        X_scaled = pd.DataFrame(self.scaler.transform(X), columns=self.columns_, index=X.index)
        return X_scaled

# ---------------------- Example Usage ----------------------

# Example column names
feature_names = [
    'torque_meas', 'outside_air_temp', 'mean_gas_temp',
    'power_avail', 'indicated_air_speed', 'net_power',
    'compressor_speed'
]

X_train = data_train.iloc[:, :-1]  # tutte le colonne tranne l'ultima → feature
y_train = data_train.iloc[:, -1]   # solo l'ultima colonna → target


pipeline = Pipeline([
    ('rename', FeatureRenamer(column_names=feature_names)),
    ('scale', DataScaler()),
])

pipeline.fit(X_train, y_train)

X_transformed = pipeline.transform(X_train)

## Decision tree prima esecuzione

In [6]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted


# ---------------------- Custom Transformers ----------------------

class FeatureRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        self._is_fitted = True  # Mark as fitted
        return self

    def _rename_df(self, df):
        df = df.drop(columns='id', errors='ignore')
        if df.shape[1] != len(self.column_names):
            raise ValueError(f"Expected {len(self.column_names)} columns, got {df.shape[1]}")
        df.columns = self.column_names
        return df

    def transform(self, X):
        check_is_fitted(self, '_is_fitted')  # Check if fit was called
        return self._rename_df(X)

class DataScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.columns_ = X.columns  # trailing underscore indicates fitted attribute
        self.scaler.fit(X)
        return self

    def transform(self, X):
        check_is_fitted(self.scaler, 'mean_')  # standard attribute for StandardScaler
        X_scaled = pd.DataFrame(self.scaler.transform(X), columns=self.columns_, index=X.index)
        return X_scaled

class DecisionTreeTrainer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = DecisionTreeClassifier(random_state=42)

    def fit(self, X, y):
        self.model.fit(X, y)
        self.is_fitted_ = True   # <-- indicatore di fit
        return self

    def transform(self, X):
        return X

    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')  # controlla se fit è stato fatto
        return self.model.predict(X)

# ---------------------- Esecuzione Pipeline ----------------------

feature_names = [
    'torque_meas', 'outside_air_temp', 'mean_gas_temp',
    'power_avail', 'indicated_air_speed', 'net_power',
    'compressor_speed'
]

X_train = data_train.iloc[:, :-1]  # tutte le colonne tranne l'ultima → feature
y_train = data_train.iloc[:, -1]   # solo l'ultima colonna → target


pipeline = Pipeline([
    ('rename', FeatureRenamer(column_names=feature_names)),
    ('scale', DataScaler()),
    ('model', DecisionTreeTrainer())
])

pipeline.fit_transform(X_train, y_train)

# Trasformiamo i dati per poterli passare alla funzione di predizione
X_transformed = pipeline.transform(X_train)

# Eseguiamo le predizioni
y_pred = pipeline.named_steps['model'].predict(X_transformed)

check_is_fitted(pipeline.named_steps['model'])

print("✔️ Pipeline fitted correttamente.")
print("Predizioni:", y_pred)
print("Feature names:", pipeline.named_steps['rename'].column_names)
print("Predizioni, numero di elementi:", len(y_pred))



✔️ Pipeline fitted correttamente.
Predizioni: [1 0 0 ... 1 1 0]
Feature names: ['torque_meas', 'outside_air_temp', 'mean_gas_temp', 'power_avail', 'indicated_air_speed', 'net_power', 'compressor_speed']
Predizioni, numero di elementi: 594100


### Pipeline tree

In [7]:
pipeline

Pipeline(steps=[('rename',
                 FeatureRenamer(column_names=['torque_meas', 'outside_air_temp',
                                              'mean_gas_temp', 'power_avail',
                                              'indicated_air_speed',
                                              'net_power',
                                              'compressor_speed'])),
                ('scale', DataScaler()), ('model', DecisionTreeTrainer())])

## K-fold

In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import cross_val_score, KFold


# ---------------------- Custom Transformers ----------------------

class FeatureRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        self._is_fitted = True  # Mark as fitted
        return self

    def _rename_df(self, df):
        df = df.drop(columns='id', errors='ignore')
        if df.shape[1] != len(self.column_names):
            raise ValueError(f"Expected {len(self.column_names)} columns, got {df.shape[1]}")
        df.columns = self.column_names
        return df

    def transform(self, X):
        check_is_fitted(self, '_is_fitted')  # Check if fit was called
        return self._rename_df(X)

class DataScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.columns_ = X.columns  # trailing underscore indicates fitted attribute
        self.scaler.fit(X)
        return self

    def transform(self, X):
        check_is_fitted(self.scaler, 'mean_')  # standard attribute for StandardScaler
        X_scaled = pd.DataFrame(self.scaler.transform(X), columns=self.columns_, index=X.index)
        return X_scaled
    
class DecisionTreeTrainer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = DecisionTreeClassifier(random_state=42)

    def fit(self, X, y):
        self.model.fit(X, y)
        self.is_fitted_ = True   # <-- indicatore di fit
        return self

    def transform(self, X):
        return X

    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')  # controlla se fit è stato fatto
        return self.model.predict(X)

class DecisionTreeCrossValidator(BaseEstimator, TransformerMixin):
    def __init__(self, n_splits=5, random_state=None, shuffle=True):
        self.n_splits = n_splits
        self.random_state = random_state
        self.shuffle = shuffle
        self.cv_scores = None
        self.model = DecisionTreeClassifier(criterion="gini", random_state=42)

    def fit(self, X, y):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        kf = KFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state=self.random_state)

        print(f"\nPerforming {self.n_splits}-Fold Cross-Validation on training data...")
        self.cv_scores = cross_val_score(
            self.model,
            X,
            y,
            cv=kf,
            scoring='precision'  # puoi cambiare scoring se vuoi
        )
        print("Cross-validation complete.")
        print(f"Cross-validation scores ({self.n_splits} folds): {self.cv_scores}")
        print(f"Mean CV score: {self.cv_scores.mean():.4f}")
        print(f"Standard deviation of CV scores: {self.cv_scores.std():.4f}")

        return self


    def transform(self, X):
        if self.cv_scores is None:
            raise RuntimeError("Cross-validation has not been performed yet. Call fit() first.")
        # Non modifichiamo X, solo ritorniamo lo stesso DataFrame per compatibilità pipeline
        return X

    def get_cv_scores(self):
        if self.cv_scores is None:
            raise RuntimeError("Cross-validation has not been performed yet. Call fit() first.")
        return self.cv_scores


# ---------------------- Example Usage ----------------------

# Example column names
new_names_for_train_data = [
    'torque_meas', 'outside_air_temp', 'mean_gas_temp',
    'power_avail', 'indicated_air_speed', 'net_power',
    'compressor_speed'
]

X_train = data_train.iloc[:, :-1]  # tutte le colonne tranne l'ultima → feature
y_train = data_train.iloc[:, -1]   # solo l'ultima colonna → target

# Pipeline
# Cross-validation pipeline (solo valutazione)
pipeline_cv = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data)),
    ('scale_data', DataScaler()),
    ('cross_validate_decision_tree', DecisionTreeCrossValidator(n_splits=5, random_state=42))
])

# Addestramento e predizione finale
pipeline_final = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data)),
    ('scale_data', DataScaler()),
    ('train_decision_tree', DecisionTreeTrainer())
])

# Esegui la validazione incrociata
pipeline_cv.fit(X_train, y_train)

# Fit finale per predizione
pipeline_final.fit(X_train, y_train)

# Predizione
y_pred = pipeline_final.named_steps['train_decision_tree'].predict(
    pipeline_final.named_steps['scale_data'].transform(
        pipeline_final.named_steps['rename_features'].transform(X_train)
    )
)

# Verifiche
check_is_fitted(pipeline_final.named_steps['train_decision_tree'])
print("✔️ Pipeline fitted correttamente.")
print("Predizioni:", y_pred)
print("Numero predizioni:", len(y_pred))
print("Feature names:", pipeline_final.named_steps['rename_features'].column_names)


Performing 5-Fold Cross-Validation on training data...
Cross-validation complete.
Cross-validation scores (5 folds): [0.99586397 0.99588101 0.99645925 0.99556749 0.9958109 ]
Mean CV score: 0.9959
Standard deviation of CV scores: 0.0003
✔️ Pipeline fitted correttamente.
Predizioni: [1 0 0 ... 1 1 0]
Numero predizioni: 594100
Feature names: ['torque_meas', 'outside_air_temp', 'mean_gas_temp', 'power_avail', 'indicated_air_speed', 'net_power', 'compressor_speed']


## Performance

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ---------------------- Custom Transformers ----------------------

class FeatureRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        if not isinstance(column_names, list):
            raise TypeError("column_names must be a list.")
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def _rename_df(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError("Each input must be a DataFrame.")

        df = df.drop(columns='id', errors='ignore')

        if len(self.column_names) != df.shape[1]:
            if 'faulty' in df.columns and len(self.column_names) + 1 == df.shape[1]:
                print("Detected 'faulty' column, adjusting rename.")
                df.columns = self.column_names + ['faulty']
            else:
                raise ValueError("Column count mismatch after dropping 'id'.")
        else:
            df.columns = self.column_names

        return df

    def transform(self, X):
        if isinstance(X, tuple):
            return tuple(self._rename_df(df) for df in X)
        else:
            return self._rename_df(X)


class DataScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.feature_columns = None
        self.target_column = None

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        self.target_column = X.columns[-1]
        self.feature_columns = X.columns[:-1]

        self.scaler.fit(X[self.feature_columns])
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        X_scaled = X.copy()
        X_scaled[self.feature_columns] = self.scaler.transform(X[self.feature_columns])

        return X_scaled


class DecisionTreeTrainer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = DecisionTreeClassifier(criterion="gini", random_state=42)
        self.feature_columns = None
        self.target_column = None

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        self.target_column = X.columns[-1]
        self.feature_columns = X.columns[:-1]

        X_train = X[self.feature_columns]
        y_train = X[self.target_column]

        print(f"Training Decision Tree Classifier with criterion='{self.model.criterion}'...")
        self.model.fit(X_train, y_train)
        print("Decision Tree model training complete.")

        return self

    def transform(self, X):
        # Qui trasformiamo X restituendo X senza modifiche,
        # oppure potresti anche ritornare solo il modello, dipende da cosa ti serve.
        # Per mantenere compatibilità con pipeline, ritorniamo il DataFrame non modificato.
        return X

    def predict(self, X):
        if self.model is None:
            raise RuntimeError("Model has not been trained yet. Call fit() first.")

        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")

        X_feat = X[self.feature_columns]
        return self.model.predict(X_feat)

class DecisionTreeCrossValidator(BaseEstimator, TransformerMixin):
    def __init__(self, n_splits=5, random_state=None, shuffle=True):
        self.n_splits = n_splits
        self.random_state = random_state
        self.shuffle = shuffle
        self.cv_scores = None
        self.model = DecisionTreeClassifier(criterion="gini", random_state=42)

    def fit(self, X, y):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")
    
        # Controllo y: deve essere array-like o pandas Series
        if not (isinstance(y, (pd.Series, np.ndarray))):
            raise TypeError("Input y must be a pandas Series or numpy array.")

        kf = KFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state=self.random_state)

        print(f"\nPerforming {self.n_splits}-Fold Cross-Validation on training data...")
        self.cv_scores = cross_val_score(
            self.model,
            X,
            y,
            cv=kf,
            scoring='accuracy'  # puoi cambiare scoring se vuoi
        )
        print("Cross-validation complete.")
        print(f"Cross-validation scores ({self.n_splits} folds): {self.cv_scores}")
        print(f"Mean CV score: {self.cv_scores.mean():.4f}")
        print(f"Standard deviation of CV scores: {self.cv_scores.std():.4f}")

        # Ora allena il modello completo sui dati interi (fit finale)
        print("Training final model on the entire training set...")
        self.model.fit(X, y)
        print("Final model training complete.")

        return self


    def transform(self, X):
        if self.cv_scores is None:
            raise RuntimeError("Cross-validation has not been performed yet. Call fit() first.")
        # Non modifichiamo X, solo ritorniamo lo stesso DataFrame per compatibilità pipeline
        return X

    def get_cv_scores(self):
        if self.cv_scores is None:
            raise RuntimeError("Cross-validation has not been performed yet. Call fit() first.")
        return self.cv_scores


class PerformanceEvaluator(BaseEstimator, TransformerMixin):
    """
    Valuta le performance di un modello classificatore su un set di test.
    Salva le metriche come attributi. Funziona indipendentemente dal formato di X.
    """
    def __init__(self, trained_model=None):
        self.trained_model = trained_model
        self.accuracy = None
        self.precision = None
        self.sensitivity = None
        self.specificity = None
        self.confusion_matrix_ = None
        self.fpr = None
        self.tpr = None
        self.roc_auc = None

    def fit(self, X, y):
        if self.trained_model is None:
            raise ValueError("Il modello allenato (`trained_model`) deve essere fornito.")

        print("\nEvaluating model performance on the test set...")

        # Predict labels
        y_pred = self.trained_model.predict(X)

        # Compute metrics
        self.accuracy = accuracy_score(y, y_pred)
        self.precision = precision_score(y, y_pred, zero_division=0)
        self.sensitivity = recall_score(y, y_pred, zero_division=0)

        # Confusion matrix and specificity
        cm = confusion_matrix(y, y_pred)
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
            self.specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        else:
            self.specificity = None
        self.confusion_matrix_ = cm

        if hasattr(self.trained_model, "predict_proba"):
            y_prob = self.trained_model.predict_proba(X)[:, 1]
            self.fpr, self.tpr, _ = roc_curve(y, y_prob)
            self.roc_auc = auc(self.fpr, self.tpr)
        else:
            self.fpr = self.tpr = self.roc_auc = None

        return self

    def transform(self, X):
        # Non altera i dati, solo per compatibilità con pipeline
        return X

    def get_results(self):
        return {
            "accuracy": self.accuracy,
            "precision": self.precision,
            "sensitivity": self.sensitivity,
            "specificity": self.specificity,
            "confusion_matrix": self.confusion_matrix_,
            "fpr": self.fpr,
            "tpr": self.tpr,
            "roc_auc": self.roc_auc
        }


# Separazione delle feature e del target
X_train = data_train.iloc[:, :-1]
y_train = data_train.iloc[:, -1]

# Ricomponiamo il DataFrame per compatibilità con i tuoi transformer personalizzati
data_train_processed = X_train.copy()
data_train_processed[y_train.name] = y_train  # Aggiungiamo la colonna target in fondo

# Definizione della pipeline di training (senza PerformanceEvaluator)
pipeline = Pipeline([
    ('rename_features', FeatureRenamer(column_names=new_names_for_train_data)),
    ('scale_data', DataScaler()),
    ('train_decision_tree', DecisionTreeTrainer()),
    ('cross_validate_decision_tree', DecisionTreeCrossValidator(n_splits=5, random_state=42))
])

# 1. Applica rename_features con fit_transform
rename_step = pipeline.named_steps['rename_features']
X_train_renamed = rename_step.fit_transform(data_train_processed)

# 2. Applica scale_data con fit e transform
scale_step = pipeline.named_steps['scale_data']
scale_step.fit(X_train_renamed)
X_train_scaled = scale_step.transform(X_train_renamed)

# 3. Estrazione della variabile target
y_train = X_train_scaled['health_state']


# 4. Rimozione della variabile target dalle feature
X_scaled_only_features = X_train_scaled.drop(columns=['health_state'])

# 5. Allena il modello sui dati processati
train_step = pipeline.named_steps['train_decision_tree']
y_train = y_train.astype('int64')
train_step.fit(X_scaled_only_features, y_train)

# 6. Estrai il modello addestrato (assumo che l'attributo si chiami 'model')
trained_model = train_step.model

# 7. Valutazione delle performance sul training set
evaluator = PerformanceEvaluator(trained_model=trained_model)
evaluator.fit(X_scaled_only_features, y_train)
results = evaluator.get_results()

# 8. Stampa dei risultati
print("\nPerformance Evaluation Results:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Sensitivity (Recall): {results['sensitivity']:.4f}")
print(f"Specificity: {results['specificity']:.4f}")
print("\nConfusion Matrix:")
print(results["confusion_matrix"])

if results["roc_auc"] is not None:
    plt.figure()
    plt.plot(results["fpr"], results["tpr"], color='darkorange', lw=2, label=f'ROC curve (area = {results["roc_auc"]:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Training Set')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()
else:
    print("\nROC curve and AUC not available (predict_proba not supported).")



Training Decision Tree Classifier with criterion='gini'...


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.